 Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Выведем общую информацию о датафрейме и посмотрим на типы данных

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проверим наличие дубликатов

In [2]:
df.duplicated().sum()

54

Проверим наличие пропусков

In [3]:
df.isnull().sum()*100/len(df)

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

Посмотрим на первые и последние 5 записей

In [4]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Пробежимся по всем колонкам и найдем странности

Количество детей

In [6]:
df.sort_values('children')['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Общий трудовой стаж в днях. Отрицательные значения!

In [7]:
df[df['days_employed'] < 0]['days_employed'].count()
#df[df['days_employed'] < 0].head()

15906

Положительные значения слишком большие для количества дней, даже для людей приклонного возраста

In [8]:
#df[df['days_employed'] > 0]['days_employed'].count()
df[df['days_employed'] > 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


Возраст

In [9]:
df.sort_values('dob_years')['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Образование

In [10]:
df.sort_values('education')['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Семейное положение

In [11]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Пол

In [12]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Тип занятости

In [13]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Доход

In [14]:
# Проверим есть ли люди с нулевым уровнем дохода
df[df['total_income'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [15]:
# Проверим пропуски
df[df['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Цель получения кредита

In [16]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Наш датасет состоит из 21 525 записей;
Из них 54 дубликата – не много, однако это сырые данные и количество дубликатов может измениться после базовой предобработки;
Одинаковое количество пропусков в столбцах days_employed и total_income – 2 174. Связь между этими столбцами понятна: невозможно рассчитать общий доход не имея информации о стаже. Так как столбец total_income нам нужен для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" , то от этих записей пользы нет.
Попробуем восстановить пропуски заменив NaN медианными значениями (образование, тип дохода).
Столбец children:
Для 47 записей количество детей равно -1 – т.к. у нас нет возможности связаться с коллегами, которые сделали нам выгрузку датасета, и причину появления таких значений установить сложно.Их всего 47, и они не должны коренным образом повлиять на анализ.76 человек имеет по 20 детей. Выглядит как ошибка в данных, мы могли бы предположить, что корретное значение – это 2, а 0 появился при неправильном заполнении данных.Удаляем.
Столбец days_employed: имеет тип float64 и содержит отрицательные значения. Всего таких записей 15 906 , т.е. ~73% от общего числа!Будем работать с днями, поэтому приведем тип столбца к int64;Положительные значения из датасета выглядят странными. Например, 343937.404131 / 365 дней = 942 лет стажа.Хорошо, что в решении бизнес задачи этот столбец нам не пригодится.
А вот отрицательные значения сделаем абсолютными, т.к. эти данные выглядят правдоподобнее. Даже свою функцию писать не придется – применим стандартную abs.
Столбец dob_years – для 101 человека возраст неизвестен. Для ответа на вопросы бизнеса этот стоблец не используется. Возьмем на заметку и оставим как есть.
В столбце education встречаются категории в разном регистре – нужно будет привести к единому виду, например, все в lowercase. И еще раз проверить на дубликаты. Предварительно убедимся в том, что education_id соотносится с нужным типом образования.
В столбце family_status только одна категория Не женат / не замужем выбивается из общей семантики именования – начинается с заглавной буквы.
У одного клиента неизвестен пол – для анализа этот столбец мы не используем поэтому пока оставим эту запись;
Столбец purpose содержит разные формулировки одних и тех же целей – применим лемматизацию.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [17]:
# Меняем отрицательное значение на положительное 
df['days_employed'] = df['days_employed'].apply(abs)

In [18]:
df['education'] = df['education'].str.lower()

In [19]:
df.drop_duplicates(keep=False, inplace=True)

In [20]:
# Составим сводную таблицу, который нужена при заполнении пропусков дней стажа
median_values = pd.pivot_table(df,index=['education', 'income_type'],values=['days_employed', 'total_income'],aggfunc=np.median)

median_values

days_employed   total_income
education           income_type                                  
высшее              безработный      395302.838654  202722.511368
                    госслужащий        2531.034209  172511.107016
                    компаньон          1454.659104  201785.400018
                    пенсионер        366158.526428  144240.768611
                    предприниматель     520.848083  499163.144947
                    сотрудник          1556.457266  165640.744634
                    студент             578.751554   98201.625314
начальное           госслужащий        2787.767403  148339.290825
                    компаньон          1151.634460  136798.905143
                    пенсионер        360264.985350  102598.653164
                    сотрудник          1197.176853  125994.910603
неоконченное высшее госслужащий        1885.183639  160592.345303
                    компаньон          1017.888120  179867.152890
                    пенсионер        372250.501660  120136.896353
                    сотрудник          1197.676879  151308.937846
среднее             безработный      337524.466835   59956.991984
                    в декрете          3296.759962   53829.130729
                    госслужащий        2857.770974  136652.970357
                    компаньон          1670.809068  159070.690289
                    пенсионер        365025.338867  114842.854099
                    сотрудник          1613.581420  136555.108821
ученая степень      госслужащий        5968.075884  111392.231107
                    пенсионер        356930.517546  177088.845999
                    сотрудник          2351.431934  198570.757322

In [21]:
# Напишем функцию заполнения пропусков дней стажа на основе сводной таблицы
def fill_days_employed(row):
    return median_values.loc[row['education']].loc[row['income_type']]['days_employed']

In [22]:
# Восстановим пропуски при помощи вычисленных значений
df.loc[df['days_employed'].isnull(), 'days_employed'] = df.apply(fill_days_employed, axis=1)

In [23]:
#Напишем функцию заполнения пропусков уровня дохода на основе сводной таблицы
def fill_total_income(row):
    return median_values.loc[row['education']].loc[row['income_type']]['total_income']

In [24]:
# Восстановим пропуски при помощи вычисленных значений
df.loc[df['total_income'].isnull(), 'total_income'] = df.apply(fill_total_income, axis=1)

In [25]:
# Избавляемся от записей, в которых есть ошибочные данные
df.loc[df['children'] == 20, 'children'] = 2

In [26]:
# Избавляемся от записей, в которых есть ошибочные данные
df.loc[df['children'] == -1, 'children'] = 1

In [27]:
df.sort_values('children')['children'].value_counts().sort_index()

0    14038
1     4845
2     2125
3      330
4       41
5        9
Name: children, dtype: int64

In [28]:
df.shape[0]

21388

Мы избавились от пропусков, оставив только необходимые для будущего анализа данные

### Замена типа данных

In [29]:
df['days_employed'] = df['days_employed'].astype('int').apply(abs)

In [30]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Преобразовали количество дней стажа в целочисленный тип при помощи метода astype, дополнительно столбец избавился от отрицательных значений.

### Обработка дубликатов

In [31]:
# Приведем значения в столбце Уровень образования клиента к единому регистру
df['education'] = df['education'].str.lower()

In [32]:
df.drop_duplicates()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [33]:
df.sort_values('education')['education'].value_counts()

среднее                15116
высшее                  5240
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [34]:
df.duplicated().sum() 

0

### Вывод

После нормализации значений колонки education необходимость в удалении дубликатов при помощи метода drop_duplicates() отпала. Их количество равно 0.

### Лемматизация

In [35]:
from collections import Counter

# Воспользуемся библиотекой Mystem от Яндекса
from pymystem3 import Mystem
m = Mystem()

def purpose_category(lemmas):
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return('недвижимость')
    if 'автомобиль' in lemmas:
        return('автомобиль')
    if 'образование' in lemmas:
        return('образование')
    if 'свадьба' in lemmas:
        return('свадьба')
    if 'ремонт' in lemmas:
        return('ремонт')
    return('иное')
print(purpose_category(m.lemmatize('образование')))#проверка работы функции
df['purpose_category'] = df['purpose'].apply(m.lemmatize).apply(purpose_category)#добавляем новый стобец с категориямицелей кредита в таблицу
print()
print(df['purpose_category'].value_counts())

образование

недвижимость    10784
автомобиль       4297
образование      4004
свадьба          2303
Name: purpose_category, dtype: int64


### Вывод

Из большого числа вариаций получили всего четыре категорий целей кредита – читать и анализировать такие данные намного проще.

### Категоризация данных

In [36]:
#Распределим клиентов по наличию детей:0 – бездетная, 1 – однодетная, 2 – малодетная, 3+ – многодетная
def categorize_children(count):
    categories = {0: 'бездетная', 1: 'один ребенок', 2: 'два ребенка'}
    return categories.get(count, 'многодетная')

In [37]:
df['children_category'] = df['children'].apply(categorize_children)

In [38]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,один ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,один ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,бездетная
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,многодетная
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,бездетная


In [39]:
#По семейному положению всего у нас пять категорий, но давайте сузим запрос и возьмем за основу family_status_id до следующих категории:
   
def categorize_family_status(value):
    if value == 'женат / замужем':
        return 'в браке'
    if value == 'гражданский брак':
        return 'в г/б'
    if value == 'вдовец / вдова':
        return 'вдовцы'
    return 'не в браке' 

In [40]:
df['family_status_category'] = df['family_status'].apply(categorize_family_status)

In [41]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,один ребенок,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,один ребенок,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,бездетная,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,многодетная,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,бездетная,в г/б


In [42]:
#Проаеряем как уровень дохода влияет на возврат кредита в срок
ti_median = df['total_income'].median()
ti_median

143904.60264989812

In [43]:
df['total_income'].max()

2265604.028722744

In [44]:
def categorize_total_income(value):
    if value < 50_000:
        return 'низкий'
    elif value <= ti_median:
        return 'средний'
    elif value < 350_000:
        return 'выше среднего'
    else:
        return 'высокий'

In [45]:
df['total_income_category'] = df['total_income'].apply(categorize_total_income)

In [46]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,family_status_category,total_income_category
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,недвижимость,один ребенок,в г/б,выше среднего
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,автомобиль,бездетная,в браке,выше среднего
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,недвижимость,один ребенок,в г/б,средний
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,автомобиль,многодетная,в браке,выше среднего
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля,автомобиль,два ребенка,в браке,средний


### Вывод

Мы категоризовали данные. Теперь, группируя данные по нашим категориям, мы сможем ответить на вопросы ниже

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
children_category_addication = df.pivot_table(index='children_category', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
children_category_addication.columns = ['no debt', 'debt']
children_category_addication['%'] = children_category_addication['debt'] / (children_category_addication['debt'] + children_category_addication['no debt'])
children_category_addication.style.format({'%':'{:.2%}', 'no debt': '{:.0f}'})

,no debt,debt,%
children_category,,,
бездетная,12975,1063,7.57%
два ребенка,1923,202,9.51%
многодетная,349,31,8.16%
один ребенок,4400,445,9.18%


### Вывод

Прямой зависимости между наличием детей и возвратом кредита в срок нет: до 9.4% клиентов имели задолжности по кредитам, независимо от того были у них дети или нет.
Бездетные семьи чаще выплачивают кредит : ~7.5%.
Что интересно, многодетные семьи реже остаются в должниках ~8.2%, чем семьи с одним ребенком ~9.2% или двумя ~9.4%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
family_status_addication = df.pivot_table(index='family_status_category', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
family_status_addication.columns = ['no debt', 'debt']
family_status_addication['%'] = family_status_addication['debt'] / (family_status_addication['debt'] + family_status_addication['no debt'])
family_status_addication.style.format({'%':'{:.2%}', 'no debt': '{:.0f}'})

,no debt,debt,%
family_status_category,,,
в браке,11369,931,7.57%
в г/б,3740,388,9.40%
вдовцы,895,63,6.58%
не в браке,3643,359,8.97%


### Вывод

Клиенты, вдовцы, реже остаются в должниках. Так же клиенты состоящие в браке тоже чаще остальных отдают долги. Самые не стабильные оказались те, что в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
total_income_addication = df.pivot_table(index='total_income_category', columns = 'debt', values = 'total_income',aggfunc = 'count').fillna(0)
total_income_addication.columns = ['no debt', 'debt']
total_income_addication['%']= total_income_addication['debt'] / (total_income_addication['debt']+total_income_addication['no debt'])
total_income_addication.style.format({'%':'{:.2%}', 'no debt': '{:.0f}'})

,no debt,debt,%
total_income_category,,,
высокий,804,55,6.40%
выше среднего,9062,773,7.86%
низкий,349,23,6.18%
средний,9432,890,8.62%


### Вывод

Возврат кредита зависит от уровня дохода клиента: чем выше доход, тем меньше % задолжностей. Но есть исключение. Люди с низким уровнем дохода имеют самый низкий процент задолжностей по кредитам: ~6.2%, а вот люди со средним – самый высокий ~8.5%.

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
purpose_addication = df.pivot_table(index='purpose_category', columns = 'debt', values = 'total_income',aggfunc = 'count').fillna(0)
purpose_addication.columns = ['no debt', 'debt']
purpose_addication['%'] = purpose_addication['debt'] / (purpose_addication['debt'] + purpose_addication['no debt'])
purpose_addication.style.format({'%':'{:.2%}', 'no debt': '{:.0f}'})

,no debt,debt,%
purpose_category,,,
автомобиль,3894,403,9.38%
недвижимость,10002,782,7.25%
образование,3634,370,9.24%
свадьба,2117,186,8.08%


### Вывод

Посчитав % задолжностей по категориям, можно сделать вывод, что автомобиль и образование не на первом месте в списке приоритетов на выплаты у клиентов, в отличие от жилья, недвижимости и свадьбы.

### Шаг 4. Общий вывод

Исходя из предыдущих выводов, мы можем сделать следующую рекомедацию бизнесу по выдаче кредита: безопаснее всего выдавать кредиты на покупку жилья, недвижимости или свадьбу – по ним меньшее количество задолжностей.